In [1]:
import pandas as pd

### Load dataframe

In [2]:
df = pd.read_csv('output/simulation.tsv', sep='\t')

In [7]:
df.head()

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
0,environment,L_lactis,M_lcts_e,0.185394,0.541619,1.00,kefir,MILK
1,L_lactis,environment,M_gal_e,0.175781,0.975720,0.03,kefir,MILK
2,L_kefiranofaciens,environment,M_ac_e,0.097582,1.652698,1.00,kefir,MILK
3,L_lactis,environment,M_co2_e,0.066902,1.520169,1.00,kefir,MILK
4,environment,L_mesenteroides,M_lcts_e,0.064962,0.189784,0.94,kefir,MILK


### Analyse results

Let's first check for the main interactions highlighted in **Fig. 5e** the paper

![Fig 5e part1](output/fig5e1.png)

*L. kefiranofaciens*  *"gives"* amino acids to *L. mesenteroides*, but this is due to extracellular proteolytic activity, and we do not observe any actual cross-feeding: 

In [22]:
df.query('donor == "L_kefiranofaciens" and receiver == "L_mesenteroides" and frequency > 0.1')

,donor,receiver,compound,mass_rate,rate,frequency,community,medium


*L. mesenteroides* is expected to donate lactate to *L. kefiranofaciens*. In the simulations we observe exchange of acetaldehyde instead (this is likely due to a common issue with the models generated with CarveMe, they often ferment acetaldehyde instead of acetate and lactate).

In [10]:
df.query('receiver == "L_kefiranofaciens" and donor == "L_mesenteroides" and frequency > 0.1' )

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
15,L_mesenteroides,L_kefiranofaciens,M_acald_e,0.014656,0.332695,0.51,kefir,MILK
122,L_mesenteroides,L_kefiranofaciens,M_h2s_e,0.000207,0.006081,1.00,kefir,MILK
156,L_mesenteroides,L_kefiranofaciens,M_nh4_e,0.000053,0.002953,0.23,kefir,MILK


![Fig 5e part1](output/fig5e1.png)

*L. lactis* is expected to donate lactate, GABA and some amino acids to *A. fabarum*. In the simulations we observe the exchange of GABA (M_4abut_e). We also observe exchange of glycerol and arginine, although these were not observed experimentally.

In [11]:
df.query('donor == "L_lactis" and receiver == "A_fabarum" and frequency > 0.1')

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
30,L_lactis,A_fabarum,M_4abut_e,0.004531,0.043942,0.36,kefir,MILK
44,L_lactis,A_fabarum,M_glyc_e,0.001961,0.021295,0.99,kefir,MILK
130,L_lactis,A_fabarum,M_arg__L_e,0.000130,0.000739,0.78,kefir,MILK


No exchange was observed experimentally from *A. fabarum* to *L. lactis*, and indeed none are predicted in the simulations either:

In [14]:
df.query('donor == "A_fabarum" and receiver == "L_lactis" and frequency > 0.1')

,donor,receiver,compound,mass_rate,rate,frequency,community,medium


### Network visualization

Let's look at all potential cross-feeding interactions (with a threshold of 10% frequency in the sampling, and a mass rate of at least 0.001 gr/gDW/h). 

In [16]:
from pyvis.network import Network

net = Network(directed=True, notebook=True, height='500px', width='800px')

selected = df.query("frequency > 0.1 and mass_rate > 0.001 and donor != 'environment' and receiver != 'environment'")

species = set(selected['donor']) | set(selected['receiver'])
net.add_nodes(species)

for cpd in set(selected['compound']):
    net.add_node(cpd, shape='box')

for _, row in selected.iterrows():
    net.add_edge(row['donor'], row['compound'], value=row['mass_rate'])
    net.add_edge(row['compound'], row['receiver'], value=row['mass_rate'])

net.show('output/network.html')

output/network.html


----------
### Comparing with previous results obtained with SMETANA

Just for reference... (to remove for the course)

In [17]:
df2 = pd.read_csv('output/smetana.tsv', sep='\t')

In [18]:
df2.query('donor == "L_kefiranofaciens" and receiver == "L_mesenteroides" and smetana > 0.1')

,community,medium,receiver,donor,compound,scs,mus,mps,smetana
35,kefir,MILK,L_mesenteroides,L_kefiranofaciens,M_acald_e,NaN,0.17,1,0.17
38,kefir,MILK,L_mesenteroides,L_kefiranofaciens,M_glc__D_e,NaN,0.45,1,0.45
39,kefir,MILK,L_mesenteroides,L_kefiranofaciens,M_h2s_e,NaN,0.82,1,0.82


In [19]:
df2.query('receiver == "L_kefiranofaciens" and donor == "L_mesenteroides" and smetana > 0.1' )

,community,medium,receiver,donor,compound,scs,mus,mps,smetana
8,kefir,MILK,L_kefiranofaciens,L_mesenteroides,M_acald_e,NaN,0.29,1,0.29
9,kefir,MILK,L_kefiranofaciens,L_mesenteroides,M_h2s_e,NaN,0.88,1,0.88


In [20]:
df2.query('donor == "L_lactis" and receiver == "A_fabarum" and smetana > 0.1')

,community,medium,receiver,donor,compound,scs,mus,mps,smetana
61,kefir,MILK,A_fabarum,L_lactis,M_4abut_e,NaN,0.18,1,0.18
62,kefir,MILK,A_fabarum,L_lactis,M_acald_e,NaN,0.18,1,0.18
64,kefir,MILK,A_fabarum,L_lactis,M_glyc_e,NaN,0.18,1,0.18
65,kefir,MILK,A_fabarum,L_lactis,M_h2s_e,NaN,0.75,1,0.75


In [21]:
from pyvis.network import Network

net = Network(directed=True, notebook=True, height='500px', width='800px')

selected = df2.query("smetana > 0.1")

species = set(selected['donor']) | set(selected['receiver'])
net.add_nodes(species)

for cpd in set(selected['compound']):
    net.add_node(cpd, shape='box')

for _, row in selected.iterrows():
    net.add_edge(row['donor'], row['compound'], value=row['smetana'])
    net.add_edge(row['compound'], row['receiver'], value=row['smetana'])

net.show('output/network2.html')

output/network2.html
